<a href="https://colab.research.google.com/github/ylkim1947/human-like-summarization-translation/blob/main/main_yl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAPI를 이용한 논문, 기사, 글 요약 및 번역

https://jehyunlee.github.io/2022/07/02/Python-DS-106-aaicon/

### Reference

```
이제현, 유시현, 김창기, 김현구, "Open API를 활용한 고속 논문 분석",
실용인공지능학회지 vol.1 p.9, 2022
```

## API 키 발급

- RapidAPI : https://rapidapi.com/developer/new
- 네이버 : https://developers.naver.com/apps/#/register?api=ppg_n2mt

In [1]:
rapidapi_key = '9a44300dfdmshaa62da9bf11b6a1p1b39c8jsnfecd9f533d4e'
naver_client_id = 'br4sQgGhMNv4SSizUKgs'
naver_client_secret = 'X1dTly_NoF'

# TLDRThis

- https://rapidapi.com/tldrthishq-tldrthishq-default/api/tldrthis/

---

## 요약의 종류

1. Abstractive(Human-like) summarization

> Abstractive summarization(생성 요약)은 기존 Input text를 그대로 인용하지 않고, 기존의 내용을 새롭게 re-phrasing 하여 Summary를 생성하는 요약 모델입니다.

2. Extractive summarization

> 반면에 Extractive summarization(추출 요약)은 기존 Input text에 존재하는 중요한 단어를 그대로 사용하여 Summary를 생성하는 요약 모델입니다.

출처 : https://supkoon.tistory.com/40

In [2]:
import requests
from pprint import pprint

### 예시 논문

Attention Is All You Need
- 페이지 : https://arxiv.org/abs/1706.03762
- 본문(pdf) : https://arxiv.org/pdf/1706.03762.pdf

## 요약

In [7]:
url = "https://tldrthis.p.rapidapi.com/v1/model/abstractive/summarize-url/"

payload = {
    "url": "https://sci-hub.mksa.top/https://www.frontiersin.org/articles/10.3389/fnhum.2018.00505/full", # 주소
    "min_length": 100, # 최소 길이
    "max_length": 300, # 최대 길이
    "is_detailed": False # 한 문장으로 반환할 것인지 여부
}

headers = {
    "content-type": "application/json",
    "X-RapidAPI-Key": rapidapi_key,
    "X-RapidAPI-Host": "tldrthis.p.rapidapi.com"
}

response = requests.request("POST", url, json=payload, headers=headers)

pprint(response.json())

{'article_abstract': None,
 'article_authors': [],
 'article_html': '<p>Pinti, P., Scholkmann, F., Hamilton, A., Burgess, P., '
                 '&amp; Tachtsidis, I. (2019).  '
                 'doi:10.3389/fnhum.2018.00505</p>',
 'article_image': None,
 'article_pub_date': 'Fri, 04 Sep 2020 00:00:00 GMT',
 'article_text': 'Pinti, P., Scholkmann, F., Hamilton, A., Burgess, P., & '
                 'Tachtsidis, I. (2019). doi:10.3389/fnhum.2018.00505',
 'article_title': 'Current Status and Issues Regarding Pre-processing of fNIRS '
                  'Neuroimaging Data: An Investigation of Diverse Signal '
                  'Filtering Methods Within a General Linear Model Framework. '
                  'Frontiers in Human Neuroscience, 12.',
 'article_url': 'https://sci-hub.mksa.top/https://www.frontiersin.org/articles/10.3389/fnhum.2018.00505/full',
 'summary': ['Pinti, P. Scholkmann, F., Hamilton, A., Burgess, P., & '
             'Tachtsidis, I. (2019)doi:10.3389/fnhum.2018.00505, '


In [8]:
summary = response.json()['summary'][0].strip()

print(summary)

Pinti, P. Scholkmann, F., Hamilton, A., Burgess, P., & Tachtsidis, I. (2019)doi:10.3389/fnhum.2018.00505, http://www.nfh.org.uk/news/features/features-top-stories/top-10-stories-of-the-year.html.


## 번역

네이버 파파고 API

In [9]:
url = "https://openapi.naver.com/v1/papago/n2mt"

payload = {
    "source": "en",
    "target": "ko",
    "text": summary,
}

headers = {
    "content-type": "application/json",
    "X-Naver-Client-Id": naver_client_id,
    "X-Naver-Client-Secret": naver_client_secret,
}

response = requests.request("POST", url, json=payload, headers=headers)

pprint(response.json())

{'message': {'@service': 'naverservice.nmt.proxy',
             '@type': 'response',
             '@version': '1.0.0',
             'result': {'engineType': 'N2MT',
                        'srcLangType': 'en',
                        'tarLangType': 'ko',
                        'translatedText': 'Pinti, P. Scholkmann, F., Hamilton, '
                                          'A., Burgess, P. & Tachtsidis, I. '
                                          '(2019) doi:10.3389/fnhum. '
                                          '2018.00505, '
                                          'http://www.nfh.org.uk/news/features/features-top-stories/top-10-stories-of-the-year.html.'}}}


In [10]:
print(response.json()['message']['result']['translatedText'])

Pinti, P. Scholkmann, F., Hamilton, A., Burgess, P. & Tachtsidis, I. (2019) doi:10.3389/fnhum. 2018.00505, http://www.nfh.org.uk/news/features/features-top-stories/top-10-stories-of-the-year.html.


## 함수화

In [11]:
def summarize_and_translate(article_url, min_length=100, max_length=300):
    url = "https://tldrthis.p.rapidapi.com/v1/model/abstractive/summarize-url/"

    payload = {
        "url": article_url, # 주소
        "min_length": min_length, # 최소 길이
        "max_length": max_length, # 최대 길이
        "is_detailed": False # 한 문장으로 반환할 것인지 여부
    }

    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": rapidapi_key,
        "X-RapidAPI-Host": "tldrthis.p.rapidapi.com"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    summary = response.json()['summary'][0].strip()
    
    url = "https://openapi.naver.com/v1/papago/n2mt"

    payload = {
        "source": "en",
        "target": "ko",
        "text": summary,
    }

    headers = {
        "content-type": "application/json",
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }

    response = requests.request("POST", url, json=payload, headers=headers)
    
    return response.json()['message']['result']['translatedText']

In [12]:
summarize_and_translate("https://arxiv.org/pdf/1706.03762.pdf", 50, 100)

'지배적인 시퀀스 변환 모델은 인코더와 디코더를 포함하는 복잡한 순환 또는 컨볼루션 신경망을 기반으로 한다. 우리는 반복과 컨볼루션을 완전히 배제하고 주의 메커니즘만을 기반으로 하는 새로운 단순한 네트워크 아키텍처를 제안한다. 두 가지 기계 번역 과제를 대상으로 실험한 결과, 트랜스포머 모델은 품질이 우수함과 동시에 병렬 처리성이 높아져 학습에 훨씬 적은 시간이 소요되는 것으로 나타났습니다.'

In [13]:
summarize_and_translate("https://arxiv.org/pdf/1905.11946.pdf", 100, 200)

'컨볼루션 신경망(ConvNets)은 일반적으로 고정된 리소스 예산으로 개발된 다음 더 많은 리소스를 사용할 수 있는 경우 더 나은 정확도를 위해 확장된다. 이전 작업에서는 깊이, 너비, 이미지 크기 등 세 가지 차원 중 하나만 축척하는 것이 일반적이다. 우리는 간단하지만 매우 효과적인 복합 계수를 사용하여 깊이/폭/해상도의 모든 차원을 균일하게 확장하는 새로운 스케일링 방법을 제안한다.'